In [1]:
import matplotlib as mpl
mpl.use('Agg')
import pynbody
import matplotlib.pyplot as plt
import numpy as np
import pynbody.plot as pp
import pynbody.filt as filt
import pickle
import pandas as pd
import logging
from pynbody import array,filt,units,config,transformation
from pynbody.analysis import halo

In [2]:
pynbody.config['halo-class-priority'] =  [pynbody.halo.ahf.AHFCatalogue,
                                          pynbody.halo.GrpCatalogue,
                                          pynbody.halo.AmigaGrpCatalogue,
                                          pynbody.halo.legacy.RockstarIntermediateCatalogue,
                                          pynbody.halo.rockstar.RockstarCatalogue,
                                          pynbody.halo.subfind.SubfindCatalogue, pynbody.halo.hop.HOPCatalogue]

In [5]:
filepath = '/home/akinshol/Data/Sims/newh329/h329.cosmo50PLK.3072gst5HbwK1BH.004096'

#only available when run on Quirm
#filepath = '/home/christenc/Data/Sims/h329.cosmo50PLK.3072g/h329.cosmo50PLK.3072gst5HbwK1BH/snapshots/h329.cosmo50PLK.3072gst5HbwK1BH.004096'
halo_nums = [7]

In [21]:
print('Running for simulation %s ' % filepath)
ZSOLAR = 0.0130215
XSOLO = 0.84E-2 #What pynbody uses
XSOLH = 0.706

# first, load in the simulation and halos
s = pynbody.load(filepath)
s.physical_units()
h = s.halos()

id = []
for i in halo_nums:
    id.append(h[i].properties['#ID'])

print('Loaded simulation')

X1 = h[1].properties['Xc']
Y1 = h[1].properties['Yc']
Z1 = h[1].properties['Zc']

halo_num = halo_nums[0]

# we load the copy of the halo to minimize computational stress
halo = h.load_copy(halo_num)
halo.physical_units()

###################################
# here is where you compute your stuff
# i.e. calculate the color, gas fraction, mass of each halo
# so in this example we will just compute the stellar mass and the gas mass


hostHalo = h[halo_num].properties['hostHalo'] #will equal -1 for isolated/field galaxies and not -1 if it is a satellite

# Masses and Numbers of Particles
npart = len(halo)
nstar = len(halo.star)
ngas  = len(halo.gas)
mstar = np.sum(halo.star['mass'])
mgas = np.sum(halo.gas['mass'])
totalmass = np.sum(halo['mass'])
ovdens = h[halo_num].properties['ovdens']

print('Halo %s, %s particles' % (halo_num,npart))

# Temperature of Gas
gas_temp = np.sum(halo.gas['temp']*halo.gas['mass'])/np.sum(halo.gas['mass'])


Rvir = h[halo_num].properties['Rvir']

if halo_num==1:
        Xc = X1
        Yc = Y1
        Zc = Z1
else:
        Xc = h[halo_num].properties['Xc']
        Yc = h[halo_num].properties['Yc']
        Zc = h[halo_num].properties['Zc']

print('\t Rvir %s' % Rvir)

#Virial radius of host halo and distance to host halos
print(hostHalo)
print(halo_num)
print(halo_nums)
id = np.array(id)
print(id)
print(id==hostHalo)
if hostHalo<np.min(id):
    hostVirialR = None
    hostDist = None
else:
    hostHaloid = np.array(np.array(halo_nums)[id==hostHalo])[0]
    hostVirialR = h[hostHaloid].properties['Rvir']
    hostDist = np.sqrt((h[hostHaloid].properties['Xc'] - h[halo_num].properties['Xc'])**2 + (h[hostHaloid].properties['Yc'] - h[halo_num].properties['Yc'])**2 + (h[hostHaloid].properties['Zc'] - h[halo_num].properties['Zc'])**2)

#Gas Outflows
#center on the halos
if len(halo.gas)==0:
    goutflow1 = 0
    goutflow2 = 0
    ginflow1 = 0
    ginflow2  = 0
    ginflow0 = 0
    goutflow0 = 0
    Gout_T = 0
    Gin_T = 0
else:
    try:
        pynbody.analysis.halo.center(halo)

        #select the particles in a shell from 0.2 Rvir to 0.3 Rvir
        inner_sphere2 = pynbody.filt.Sphere(str(.2*h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        outer_sphere2 = pynbody.filt.Sphere(str(.3*h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        shell_part2 = halo[outer_sphere2 & ~inner_sphere2].gas

        #Perform calculations
        dL = .1*h[halo_num].properties['Rvir']/halo.properties['h']
        velocity2 = shell_part2['vel'].in_units('kpc yr**-1')
        r2 = shell_part2['pos'].in_units('kpc')
        Mg2 = shell_part2['mass'].in_units('Msol')
        r_mag2 = shell_part2['r'].in_units('kpc')

        G_in2 = []   #List of inflowing gas flux per particles at Rvir = .2-.3
        G_out2 = []   #List of outflowing gas flux per particles at Rvir = .2-.3
        vr2 = np.sum((velocity2*r2), axis=1)

        for x in range(len(vr2)):
            if vr2[x] < 0:
                gin2 = np.array(((vr2[x]/r_mag2[x])*Mg2[x])/dL)
                G_in2 = np.append(G_in2, gin2)
            else:
                gout2 = np.sum(((vr2[x]/r_mag2[x])*Mg2[x])/dL)
                G_out2 = np.append(G_out2, gout2)

        ginflow2 = np.sum(G_in2)
        goutflow2 = np.sum(G_out2)
        print("Rate of outflowing gas at R = .25*Rvir: %s" % goutflow2)

        #Select particles in a shell from .9 Rvir to 1 Rvir
        inner_sphere0 = pynbody.filt.Sphere(str(.9*h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        outer_sphere0 = pynbody.filt.Sphere(str(h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        shell_part0 = halo[outer_sphere0 & ~inner_sphere0].gas

        #Perform calculations
        velocity0 = shell_part0['vel'].in_units('kpc yr**-1')
        r0 = shell_part0['pos'].in_units('kpc')
        Mg0 = shell_part0['mass'].in_units('Msol')
        r_mag0 = shell_part0['r'].in_units('kpc')

        G_in0 = []
        G_out0 = []
        vr0 = np.sum((velocity0*r0), axis=1)

        for x in range(len(vr0)):
            if vr0[x] < 0:
                gin0 = np.array(((vr0[x]/r_mag0[x])*Mg0[x])/dL)
                G_in0 = np.append(G_in0, gin0)
            else:
                gout0 = np.sum(((vr0[x]/r_mag0[x])*Mg0[x])/dL)
                G_out0 = np.append(G_out0, gout0)

        ginflow0 = np.sum(G_in0)
        goutflow0 = np.sum(G_out0)
        print("Rate of outflowing gas at R = .95*Rvir: %s"% goutflow0)

        #Now for smaller shell

        inner_sphere1 = pynbody.filt.Sphere(str(.1*h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        outer_sphere1 = pynbody.filt.Sphere(str(.2*h[halo_num].properties['Rvir']) + ' kpc', [0,0,0])
        shell_part1 = halo[outer_sphere1 & ~inner_sphere1].gas

        #Perform calculations
        velocity1 = shell_part1['vel'].in_units('kpc yr**-1')
        r1 = shell_part1['pos'].in_units('kpc')
        Mg1 = shell_part1['mass'].in_units('Msol')
        r_mag1 = shell_part1['r'].in_units('kpc')

        G_in1 = []
        G_out1 = []
        vr1 = np.sum((velocity1*r1), axis=1)

        for x in range(len(vr1)):
            if vr1[x] < 0:
                gin1 = np.array(((vr1[x]/r_mag1[x])*Mg1[x])/dL)
                G_in1 = np.append(G_in1, gin1)
            else:
                gout1 = np.sum(((vr1[x]/r_mag1[x])*Mg1[x])/dL)
                G_out1 = np.append(G_out1, gout1)

        ginflow1 = np.sum(G_in1)
        goutflow1 = np.sum(G_out1)
        print("Rate of outflowing gas at R = .15*Rvir: %s" % goutflow1)

        #Finding the Temperature of inflowing and outlfowing gas
        
        out_T = []
        pm_out = []
        in_T = []
        pm_in = []

        VEL = halo.gas['vel'].in_units('kpc yr**-1')
        R = halo.gas['pos'].in_units('kpc')
        GM = halo.gas['mass'].in_units('Msol')
        R_mag = halo.gas['r'].in_units('kpc')
        T =halo.gas['temp']

        VR = np.sum((VEL*R), axis =1)

        for x in range(len(VR)):
            if VR[x]>0:
                gtmp = np.array(T[x]*GM[x])
                pmout = np.array(GM[x])
                out_T = np.append(out_T, gtmp)
                pm_out = np.append(pm_out, pmout)
            if VR[x]<0:
                gtm = np.array(T[x]*GM[x])
                pmin = np.array(GM[x])
                in_T = np.append(in_T, gtm)
                pm_in = np.append(pm_in, pmin)hostHalo = h[halo_num].properties['hostHalo']

        Gout_T = np.sum(out_T)/np.sum(pm_out)
        Gin_T = np.sum(in_T)/np.sum(pm_in)


    except Exception as err:
        goutflow1 = None
        goutflow2 = None
        ginflow1 = None
        ginflow2 = None
        ginflow0 = None
        goutflow0 = None
        Gout_T = None
        Gin_T = None
        print(err)


Running for simulation /home/christenc/Data/Sims/h329.cosmo50PLK.3072g/h329.cosmo50PLK.3072gst5HbwK1BH/snapshots/h329.cosmo50PLK.3072gst5HbwK1BH.004096 


OSError: File '/home/christenc/Data/Sims/h329.cosmo50PLK.3072g/h329.cosmo50PLK.3072gst5HbwK1BH/snapshots/h329.cosmo50PLK.3072gst5HbwK1BH.004096': format not understood or does not exist

In [19]:
print(Gout_T)
print(Gin_T)

12890.043011935071
11652.424118259509


In [23]:
from Star import *

/home/engelhar/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
